In [1]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [29]:
source('main.R')

df.train <- data_frame(
    BldgType       = c('A', 'A', 'A', 'B', 'B', 'C', 'C'),
    sale_price_log = c(1,   2,   3,   4,   5,   6,   7)
)

df.test <- data_frame(
    BldgType       = c('A', 'A', 'B', 'B', 'C')
)

res <- kaggle.house$trans$type2TransContainer$BldgType(df.train)

res$df.new
res$testsetTransformator(df.test)

BldgType,sale_price_log,BldgType.new
A,1,2.0
A,2,2.0
A,3,2.0
B,4,4.5
B,5,4.5
C,6,6.5
C,7,6.5


BldgType,BldgType.new
A,2.0
A,2.0
B,4.5
B,4.5
C,6.5
